<a href="https://colab.research.google.com/github/andreidm92/Agents_in_code/blob/main/practice/Lesson_5_bm25_report_extractor_detailed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 🧠 Теория: Поиск по документам с помощью BM25 в LlamaIndex

### Зачем нужен BM25Retriever?

В LlamaIndex чаще всего используют **векторный поиск** (через эмбеддинги), но есть и **лексический подход** — **BM25**.  
Этот метод не требует внешних моделей и отлично подходит для:
- финансовых или юридических документов;
- точного поиска по терминам (например, "операционные расходы").

BM25 — классическая формула из информационного поиска. Она оценивает релевантность документа на основе:
- частоты ключевого слова в документе;
- длины документа;
- количества документов, где встречается это слово.

### Сравнение с векторным поиском
| Характеристика        | Векторный поиск              | BM25 (лексический)             |
|------------------------|------------------------------|-------------------------------|
| Основа                 | Семантика (значение)         | Лексика (точные слова)        |
| Требует эмбеддинги?    | Да                           | Нет                           |
| Подходит для           | Обобщённого смысла           | Точного термина               |
| Пример                 | "чистая прибыль" ≈ "net gain"| "выручка" ≠ "прибыль"         |

### Архитектура LlamaIndex для BM25:
1. Загрузить документы `SimpleDirectoryReader`.
2. Преобразовать их в индекс (`DocumentSummaryIndex`).
3. Подключить `BM25Retriever`.
4. Отправлять запросы через `query_engine`.

👉 Подходит для задач юристов, аналитиков, бухгалтеров.


In [3]:
!pip install llama-index llama-index-retrievers-bm25 llama-index-llms-openai


In [11]:
# 📦 Импорты
from llama_index.core import SimpleDirectoryReader, DocumentSummaryIndex
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.response.notebook_utils import display_source_node
import os, getpass



In [6]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Вставь OpenAI API ключ: ")

Вставь OpenAI API ключ: ··········


In [12]:
# 📂 Шаг 1. Загрузка документов
reader = SimpleDirectoryReader("data")
docs = reader.load_data()


print(f"Загружено документов: {len(docs)}")
print("Пример содержимого:", docs[0].text[:500])

Загружено документов: 84
Пример содержимого: UNITED STATES SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
 _____________________________________________________________________
FORM 10-K
 _____________________________________________________________________
(Mark One)
☒ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the fiscal year ended December 31, 2024
OR
☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the transition period from          


### Разделение документов на узлы

In [13]:
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(docs)

### Создание BM25Retriever:

In [14]:
retriever = BM25Retriever.from_defaults(nodes=nodes, similarity_top_k=2)


DEBUG:bm25s:Building index from IDs objects


Выполнение запроса и отображение результатов:

In [15]:
query = "Revenue 2023"
results = retriever.retrieve(query)
for node in results:
    display_source_node(node)


**Node ID:** 0ad51a25-d1b6-4d77-a780-2bf88577fbc2<br>**Similarity:** 1.6230618953704834<br>**Text:** Table of Contents
Streaming Revenues
    
We primarily derive revenues from monthly membership fe...<br>

**Node ID:** f5449edd-9afa-4ad7-8330-59183862e3f1<br>**Similarity:** 1.5545902252197266<br>**Text:** Table of Contents
Latin America (LATAM)
As of/Year Ended December 31, Change
 2024 2023 2022 2024...<br>

1. 🔄 Подключить LLM для генерации ответа по результатам:

In [16]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.llms.openai import OpenAI

query_engine = RetrieverQueryEngine.from_args(retriever, llm=OpenAI())
response = query_engine.query("What was the revenue in 2023?")
print(response)


The revenue in 2023 was $33,640,458.


🔍 Retrieval ≠ обязательно векторное индексирование

Retrieval — это общий механизм "найди релевантные фрагменты", а вот:

Тип поиска	      Что используется	   

🔢 Векторный	Семантическое сходство (через эмбеддинги)	VectorStoreIndex, Пример в LlamaIndex: FAISS, Chroma, Qdrant

🔤 Лексический	Ключевые слова, Пример в LlamaIndex:TF-IDF, BM25	BM25Retriever

🧩 Структурный	Метаданные, фильтры, условия	Пример в LlamaIndex: QueryEngine с фильтрами или SQL-like

📦 Мы используем: retrieval, но не векторный.

BM25Retriever — это лексическое индексирование:

индексируются ключевые слова (tf-idf)

на запрос подбираются "похожие по словам" фрагменты


## 📘 Пояснение к коду

В версии LlamaIndex ≥ 0.12.x DocumentSummaryIndex не интегрируется напрямую с BM25Retriever — потому что BM25Retriever работает по узлам, а DocumentSummaryIndex хранит summary как один большой текстовый узел на каждый документ.

📌 Что это значит

Если ты хочешь использовать BM25Retriever, то:

тебе нужно самому извлечь узлы (например, через SentenceSplitter)

и передать их напрямую в BM25Retriever.from_defaults(nodes=...)

А DocumentSummaryIndex в этой цепочке не нужен

✅ Когда использовать DocumentSummaryIndex

Ты используешь DocumentSummaryIndex, если:

хочешь получить summary всего документа через OpenAI

и потом искать по этим summary, а не по всему содержимому

🔹 В этом случае ты используешь .as_query_engine() с OpenAI напрямую (без BM25).

### DocumentSummaryIndex + Запрос

In [19]:
# 🧠 Индексация через DocumentSummaryIndex (по summary на документ)
from llama_index.core import DocumentSummaryIndex
from llama_index.core.query_engine import RetrieverQueryEngine

docsummary_index = DocumentSummaryIndex.from_documents(docs)

# 🔌 LLM (можно указать модель, например 'gpt-3.5-turbo')
llm = OpenAI(model="gpt-3.5-turbo")

# 🔄 Получаем retriever из индекса
retriever = docsummary_index.as_retriever()

# ⚙️ Собираем Query Engine с LLM
query_engine = RetrieverQueryEngine.from_args(retriever=retriever, llm=llm)




current doc id: b3d8204f-2ac0-4fd8-ac67-f194b7ad8363
current doc id: f8cf5677-d21c-4e90-9a7d-e24e99eac4d6
current doc id: 36ec9b70-3c52-4925-8ad7-4ee2004d4548
current doc id: f841d29c-ae19-4b5f-983f-2ea84bb29567
current doc id: eee19b26-0778-494b-81a0-0332b4ce7b4b
current doc id: 8bdc7d53-a78f-4cc7-abf2-c1743b95f90c
current doc id: 45f34f29-720b-4e48-8a95-93db470763f5
current doc id: d5e50083-8b56-44e5-9b62-c17b5d3bd14f
current doc id: abc5e1e9-e5f8-4521-9927-819e4d8a56d3
current doc id: f0cda131-c227-4424-9975-a5f9afe1ce9f
current doc id: a5f65831-b72d-4d91-9f94-476a280f4c4c
current doc id: 13fad522-4e4f-4af9-91d0-553ac5cd7e42
current doc id: 7e96d02c-0413-4938-bba1-8febe7bb1231
current doc id: 4b932fe6-5f4f-4d13-9b3d-fcc2e6bbb4b8
current doc id: e48126cf-7afb-40f4-8a4e-107a47538e2f
current doc id: a79f5219-16b2-43be-9091-03da388534fb
current doc id: a0cdca05-f46c-403a-b885-48f974f64d34
current doc id: 84d54efc-ac6b-4789-b548-fea062535d92
current doc id: 6c87572e-e375-4450-9d1b-4196fe

In [20]:
# 🔍 Запрос с генерацией ответа
response = query_engine.query("Какая была выручка в 2023?")
print(response)

The revenue in 2023 was $33,723,297.


🚫 Когда нет точного ответа:

DocumentSummaryIndex индексирует только summary документов, которые:

содержат общую информацию (в духе: "Документ о финансовых результатах")

не гарантируют включения конкретных чисел

QueryEngine по умолчанию работает так:

ищет наиболее релевантный summary (по ключевым словам)

передаёт этот текст в LLM

но LLM не делает глубокого анализа PDF — он отвечает только по summary

Если LLM не нашёл в summary точной цифры, он:

отвечает расплывчато

или не отвечает вовсе